In [42]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.compose import ColumnTransformer

In [43]:
df = pd.read_csv('../house_prices_selection.csv')
print(df.shape)
df.head()


(1460, 32)


,EVI,ZoningScore,MSZoning_RL,LotConfig_CulDSac,LandContour_HLS,LotShape_IR2,Condition1_Norm,Condition1_Feedr,LotShape_Reg,MSZoning_RM,...,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,GrLivArea,TotalBsmtSF,1stFlrSF,BsmtQual,CentralAir,Foundation
0,0.437530,0.835044,True,False,False,False,True,False,True,False,...,7,2003,2003,196.0,1710,856,856,Gd,Y,PConc
1,0.364741,0.835044,True,False,False,False,False,True,True,False,...,6,1976,1976,0.0,1262,1262,1262,Gd,Y,CBlock
2,0.411955,0.835044,True,False,False,False,True,False,False,False,...,7,2001,2002,162.0,1786,920,920,Gd,Y,PConc
3,0.394783,0.835044,True,False,False,False,True,False,False,False,...,7,1915,1970,0.0,1717,756,961,TA,Y,BrkTil
4,0.399533,0.835044,True,False,False,False,True,False,False,False,...,8,2000,2000,350.0,2198,1145,1145,Gd,Y,PConc


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   EVI                1460 non-null   float64
 1   ZoningScore        1460 non-null   float64
 2   MSZoning_RL        1460 non-null   bool   
 3   LotConfig_CulDSac  1460 non-null   bool   
 4   LandContour_HLS    1460 non-null   bool   
 5   LotShape_IR2       1460 non-null   bool   
 6   Condition1_Norm    1460 non-null   bool   
 7   Condition1_Feedr   1460 non-null   bool   
 8   LotShape_Reg       1460 non-null   bool   
 9   MSZoning_RM        1460 non-null   bool   
 10  Price_per_m^2      1460 non-null   float64
 11  PDI                1460 non-null   float64
 12  SalePrice          1460 non-null   int64  
 13  LotFrontage        1201 non-null   float64
 14  LotArea            1460 non-null   int64  
 15  WoodDeckSF         1460 non-null   int64  
 16  OpenPorchSF        1460 

In [45]:
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = df.select_dtypes(include=['object']).columns.tolist()
boolean_features = df.select_dtypes(include=['bool']).columns.tolist()

print(f"Coloane numerice ({len(numerical_features)}): {numerical_features}")
print(f"\nColoane categorice ({len(categorical_features)}): {categorical_features}")   
print(f"\nColoane booleene ({len(boolean_features)}): {boolean_features}")

Coloane numerice (19): ['EVI', 'ZoningScore', 'Price_per_m^2', 'PDI', 'SalePrice', 'LotFrontage', 'LotArea', 'WoodDeckSF', 'OpenPorchSF', 'FullBath', 'GarageCars', 'GarageArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'GrLivArea', 'TotalBsmtSF', '1stFlrSF']

Coloane categorice (5): ['KitchenQual', 'PavedDrive', 'BsmtQual', 'CentralAir', 'Foundation']

Coloane booleene (8): ['MSZoning_RL', 'LotConfig_CulDSac', 'LandContour_HLS', 'LotShape_IR2', 'Condition1_Norm', 'Condition1_Feedr', 'LotShape_Reg', 'MSZoning_RM']


In [46]:
numerical_features.remove('SalePrice') 
print(f"Coloane numerice ({len(numerical_features)}): {numerical_features}")
print(f"\nColoane categorice ({len(categorical_features)}): {categorical_features}")

Coloane numerice (18): ['EVI', 'ZoningScore', 'Price_per_m^2', 'PDI', 'LotFrontage', 'LotArea', 'WoodDeckSF', 'OpenPorchSF', 'FullBath', 'GarageCars', 'GarageArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'GrLivArea', 'TotalBsmtSF', '1stFlrSF']

Coloane categorice (5): ['KitchenQual', 'PavedDrive', 'BsmtQual', 'CentralAir', 'Foundation']


In [47]:
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing_Count': missing_values.values,
    'Percentage': missing_percent.values
})
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)
print(f"\nValori lipsă în dataset:")
print(missing_df)


Valori lipsă în dataset:
         Column  Missing_Count  Percentage
13  LotFrontage            259   17.739726
29     BsmtQual             37    2.534247
25   MasVnrArea              8    0.547945


In [48]:
df_test = pd.read_csv('../data/Cornescu_Darius_selected_features.csv')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SalePrice          1460 non-null   int64  
 1   LotFrontage        1460 non-null   float64
 2   ZoningScore        1460 non-null   float64
 3   EVI                1460 non-null   float64
 4   Price_per_m^2      1460 non-null   float64
 5   LotArea            1460 non-null   int64  
 6   PDI                1460 non-null   float64
 7   MSZoning_RL        1460 non-null   bool   
 8   LotConfig_CulDSac  1460 non-null   bool   
 9   LotShape_IR2       1460 non-null   bool   
 10  LandContour_HLS    1460 non-null   bool   
 11  Condition1_Norm    1460 non-null   bool   
 12  Condition1_Feedr   1460 non-null   bool   
 13  LotShape_Reg       1460 non-null   bool   
 14  MSZoning_RM        1460 non-null   bool   
dtypes: bool(8), float64(5), int64(2)
memory usage: 91.4 KB


In [49]:
missing_values = df_test.isnull().sum()
missing_percent = (missing_values / len(df_test)) * 100
missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing_Count': missing_values.values,
    'Percentage': missing_percent.values
})
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)
print(f"\nValori lipsă în dataset:")
print(missing_df)


Valori lipsă în dataset:
Empty DataFrame
Columns: [Column, Missing_Count, Percentage]
Index: []


In [50]:
for cols in boolean_features:
    df[cols] = df[cols].astype(int)

numerical_features.extend(boolean_features)
print(f"Coloane numerice (inclusiv boolean) ({len(numerical_features)}): {numerical_features}")
print(f"\nColoane categorice ({len(categorical_features)}): {categorical_features}")

Coloane numerice (inclusiv boolean) (26): ['EVI', 'ZoningScore', 'Price_per_m^2', 'PDI', 'LotFrontage', 'LotArea', 'WoodDeckSF', 'OpenPorchSF', 'FullBath', 'GarageCars', 'GarageArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'GrLivArea', 'TotalBsmtSF', '1stFlrSF', 'MSZoning_RL', 'LotConfig_CulDSac', 'LandContour_HLS', 'LotShape_IR2', 'Condition1_Norm', 'Condition1_Feedr', 'LotShape_Reg', 'MSZoning_RM']

Coloane categorice (5): ['KitchenQual', 'PavedDrive', 'BsmtQual', 'CentralAir', 'Foundation']


In [51]:
X = df.drop('SalePrice', axis=1)
Y = df['SalePrice']
print(f"Features shape: {X.shape}")
print(f"Target shape: {Y.shape}")

Features shape: (1460, 31)
Target shape: (1460,)


In [52]:
df_clean = df.dropna()
X_clean = df_clean.drop('SalePrice', axis=1)
Y_clean = df_clean['SalePrice']
print(f"\nDataset după eliminarea valorilor lipsă: {df_clean.shape}")
print(f"Rânduri eliminate: {len(df) - len(df_clean)}")


Dataset după eliminarea valorilor lipsă: (1164, 32)
Rânduri eliminate: 296


In [53]:
numeric_features_clean = X_clean.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features_clean = X_clean.select_dtypes(include=['object']).columns.tolist()

In [54]:
preprocessor_dropna = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features_clean),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features_clean)
    ])

pipeline_dropna = Pipeline([
    ('preprocessor', preprocessor_dropna),
    ('regressor', LinearRegression())
])

X_train_dropna, X_test_dropna, Y_train_dropna, Y_test_dropna = train_test_split( X_clean, Y_clean, test_size=0.2, random_state=42)

pipeline_dropna.fit(X_train_dropna, Y_train_dropna)
Y_pred_dropna = pipeline_dropna.predict(X_test_dropna)

mae_dropna = mean_absolute_error(Y_test_dropna, Y_pred_dropna)
rmse_dropna = np.sqrt(mean_squared_error(Y_test_dropna, Y_pred_dropna))
r2_dropna = r2_score(Y_test_dropna, Y_pred_dropna)

print(f"\nREZULTATE ABORDAREA 1 (dropna):")
print(f"MAE: ${mae_dropna:,.2f}")
print(f"RMSE: ${rmse_dropna:,.2f}")
print(f"R² Score: {r2_dropna:.4f}")




REZULTATE ABORDAREA 1 (dropna):
MAE: $21,732.42
RMSE: $35,344.64
R² Score: 0.8358


In [55]:
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7)),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

preprocessor_imputer = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

pipeline_imputer = Pipeline([
    ('preprocessor', preprocessor_imputer),
    ('regressor', LinearRegression())
])

X_train_imputer, X_test_imputer, Y_train_imputer, Y_test_imputer = train_test_split( X, Y, test_size=0.2, random_state=42 )

pipeline_imputer.fit(X_train_imputer, Y_train_imputer)
y_pred_imputer = pipeline_imputer.predict(X_test_imputer)

mae_imputer = mean_absolute_error(Y_test_imputer, y_pred_imputer)
rmse_imputer = np.sqrt(mean_squared_error(Y_test_imputer, y_pred_imputer))
r2_imputer = r2_score(Y_test_imputer, y_pred_imputer)

print(f"\nREZULTATE ABORDAREA 2 (SimpleImputer):")
print(f"MAE: ${mae_imputer:,.2f}")
print(f"RMSE: ${rmse_imputer:,.2f}")
print(f"R² Score: {r2_imputer:.4f}")


REZULTATE ABORDAREA 2 (SimpleImputer):
MAE: $21,892.38
RMSE: $34,297.28
R² Score: 0.8466


**Rezultatele celor două abordări sunt apropiate, însă diferă în moduri relevante.** Abordarea 1 (dropna) oferă un MAE puțin mai bun, dar un RMSE mai slab și un scor R² mai mic. În schimb, Abordarea 2 (cu *KNNImputer*) are un MAE ușor mai mare, însă obține cel mai mic RMSE și cel mai ridicat R², indicând o capacitate mai bună de a explica variația datelor și o performanță globală mai stabilă.

**Per ansamblu, metoda cu imputare pare mai potrivită pentru acest set de date**, deoarece păstrează mai multe informații și conduce la un model cu performanță generală superioară.


# Next laboratory :
```
Azi continuam lucrul pe notebook-ul de data trecuta. Planul este urmatorul:
Incarcam datasetul cu toate datele.
Cream pipeline-uri folosind SimpleImputer si OneHotEncoder pentru variabilele categorice.
Comparam modelele Linear Regression, DecisionTreeRegressor si RandomForestRegressor.
Pe fiecare model vom aplica Cross Validation impreuna cu GridSearchCV.
```

In [56]:
numerical_transformer_decision_tree = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7))
])

categorical_transformer_decision_tree = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

preprocessor_decision_tree = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer_decision_tree, numerical_features),
        ('cat', categorical_transformer_decision_tree, categorical_features)
    ])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
models = {
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'Decision Tree': {
        'model': DecisionTreeRegressor(random_state=42),
        'params': {
            'regressor__max_depth': [5, 10, 15, 20, None],
            'regressor__min_samples_split': [2, 5, 10],
            'regressor__min_samples_leaf': [1, 2, 4, 8],
            'regressor__max_features': ['sqrt', 'log2', None],
            'regressor__criterion': ['squared_error', 'friedman_mse']
        }
    },
    'Random Forest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {
            'regressor__n_estimators': [100, 300, 500],
            'regressor__max_depth': [10, 15, 20, None],
            'regressor__min_samples_leaf': [1, 2, 4, 8],
            'regressor__min_samples_split': [2, 5, 10],
            'regressor__max_features': ['sqrt', 'log2', None],
            'regressor__bootstrap': [True]
        }
    }
}


In [ ]:
results = {}
for model_name, configurations in models.items():
    print(f"\nTunning model: {model_name}")

    pripeline = Pipeline([
        ('preprocessor', preprocessor_decision_tree),
        ('regressor', configurations['model'])
    ])

    cross_validation = cross_val_score(pripeline, X_train, Y_train, cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')
    print(f"\nCross-Validation MAE: ${-cross_validation.mean():,.2f} (+/- ${cross_validation.std():,.2f})")

    if configurations['params']:
        grid_search = GridSearchCV(pripeline, configurations['params'], cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')
        grid_search.fit(X_train, Y_train)
        best_model = grid_search.best_estimator_
        print(f"Best parameters: {grid_search.best_params_}")
        best_model = grid_search.best_estimator_
    else:
        best_model = pripeline
        best_model.fit(X_train, Y_train)
    
    Y_pred = best_model.predict(X_test)
    mae = mean_absolute_error(Y_test, Y_pred)
    rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))
    r2 = r2_score(Y_test, Y_pred)

    results[model_name] = {
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'model': best_model
    }
    
    print(f"\n--- REZULTATE FINALE ---")
    print(f"MAE: ${mae:,.2f}")
    print(f"RMSE: ${rmse:,.2f}")
    print(f"R² Score: {r2:.4f}")




Tunning model: Linear Regression

Cross-Validation MAE: $20,967.46 (+/- $2,470.91)

--- REZULTATE FINALE ---
MAE: $21,892.38
RMSE: $34,297.28
R² Score: 0.8466

Tunning model: Decision Tree

Cross-Validation MAE: $26,126.09 (+/- $3,630.17)
Best parameters: {'regressor__criterion': 'squared_error', 'regressor__max_depth': 20, 'regressor__max_features': None, 'regressor__min_samples_leaf': 4, 'regressor__min_samples_split': 10}

--- REZULTATE FINALE ---
MAE: $21,060.28
RMSE: $32,200.02
R² Score: 0.8648

Tunning model: Random Forest

Cross-Validation MAE: $17,447.79 (+/- $1,544.83)
Best parameters: {'regressor__bootstrap': True, 'regressor__max_depth': 15, 'regressor__max_features': 'sqrt', 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 500}

--- REZULTATE FINALE ---
MAE: $16,132.14
RMSE: $27,392.48
R² Score: 0.9022


| # | Model             | MAE       | RMSE      | R²     |
|---|-------------------|-----------|-----------|--------|
| 0 | Linear Regression | 21892.38  | 34297.28  | 0.8466 |
| 1 | Decision Tree     | 21060.28  | 32200.02  | 0.8648 |
| **2** | **Random Forest**     | **16132.14**  | **27392.48**  | **0.9022** |

## Comparație între modele

- **Random Forest** este modelul cu cea mai bună performanță:
  - Are **cel mai mic MAE (~16.1k)** → greșeală medie absolută cea mai mică în estimarea prețului.
  - Are **cel mai mic RMSE (~27.4k)** → penalizează mai puțin erorile mari, ceea ce arată o predicție mai stabilă.
  - Are **cel mai mare R² (~0.90)** → explică aproximativ 90% din variația prețului.

- **Decision Tree**:
  - Performanță **mai bună decât Linear Regression** (MAE și RMSE mai mici, R² mai mare).

- **Linear Regression**:
  - Este cel mai simplu și mai ușor de interpretat model, dar:
  - Are **MAE și RMSE mai mari** și **R² mai mic** față de modelele bazate pe arbori, ceea ce arată că nu surprinde suficient de bine relațiile neliniare din date.
